# Modelo para apuestas Premier League

- El predictor está basado en el Pi rating
- Primero cargamos en dataframes los datos de la temporada actual y la anterior, se usa  
  la temporada anterior para calcular los elos iniciales de la actual.

In [9]:
from typing import Tuple, List
import numpy as np
import pandas as pd
import pandasql as pdsql
import matplotlib.pyplot as plt
import sys
import math


# current_season: str = f"{sys.argv[1]}-{int(sys.argv[2]) - 2000}"
# last_season: str = f"{int(sys.argv[1]) - 1:02d}-{int(sys.argv[1]) - 2000:02d}"

first_season: str = "2005-06"
last_season: str = "2019-20"


full_df: pd.DataFrame = pd.DataFrame()
for season in range(int(first_season[:4]), int(last_season[:4]) + 1):
    season_str: str = f"{season:02d}-{season + 1 - 2000:02d}"
    season_df: pd.DataFrame = pd.read_csv(f"Datasets/premier/{season_str}.csv")
    season_df["Season"] = season_str
    full_df = pd.concat([full_df, season_df])
full_df.reset_index(drop=True, inplace=True)
full_df.head(20)


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,13/08/05,Aston Villa,Bolton,2,2,D,2,2,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,13/08/05,Everton,Man United,0,2,A,0,1,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,13/08/05,Fulham,Birmingham,0,0,D,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,13/08/05,Man City,West Brom,0,0,D,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,13/08/05,Middlesbrough,Liverpool,0,0,D,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,E0,13/08/05,Portsmouth,Tottenham,0,2,A,0,1,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,E0,13/08/05,Sunderland,Charlton,1,3,A,1,1,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,E0,13/08/05,West Ham,Blackburn,3,1,H,0,1,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,E0,14/08/05,Arsenal,Newcastle,2,0,H,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,E0,14/08/05,Wigan,Chelsea,0,1,A,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Definimos funciones  para manipular rating.

In [10]:
LOG_BASE: int = 10
ERROR_CONSTANT: int = 3
HOME_PERFORMANCE_LEARNING_RATE: int = 0.7 # \gamma
GOAL_PERFORMANCE_LEARNING_RATE: int = 0.035 # \lambda

def get_new_pi_rating(rating_H_H: int, rating_H_A: int, rating_A_H: int, rating_A_A: int, score_H: int, score_A: int) -> Tuple[int, int, int, int]:
  expected_goal_diff = get_expected_goal_diff(rating_H_H, rating_A_A)
  observed_goal_diff = score_H - score_A
  error = abs(expected_goal_diff - observed_goal_diff)
  weighted_error_H = weighting_error(error) if expected_goal_diff < observed_goal_diff else -weighting_error(error)
  weighted_error_A = weighting_error(error) if expected_goal_diff > observed_goal_diff else -weighting_error(error)

  new_rating_H_H = rating_H_H + weighted_error_H * GOAL_PERFORMANCE_LEARNING_RATE
  new_rating_H_A = rating_H_A + (new_rating_H_H - rating_H_H) * HOME_PERFORMANCE_LEARNING_RATE
  new_rating_A_A = rating_A_A + weighted_error_A * GOAL_PERFORMANCE_LEARNING_RATE
  new_rating_A_H = rating_A_H + (new_rating_A_A - rating_A_A) * HOME_PERFORMANCE_LEARNING_RATE

  return (new_rating_H_H, new_rating_H_A, new_rating_A_H, new_rating_A_A)

def get_expected_goal_diff(rating_h_h, rating_a_a) -> float:
  if (rating_h_h >= 0):
    expected_goals_h = 10 ** np.abs((rating_h_h / ERROR_CONSTANT)) - 1
  else:
    expected_goals_h = - (10 ** np.abs((rating_h_h / ERROR_CONSTANT)) - 1)
  if (rating_a_a >= 0):
    expected_goals_a = 10 ** np.abs((rating_a_a / ERROR_CONSTANT)) - 1
  else:
    expected_goals_a = - (10 ** np.abs((rating_a_a / ERROR_CONSTANT)) - 1)
  
  return expected_goals_h - expected_goals_a



def weighting_error(goal_diff: int) -> float:
  return ERROR_CONSTANT * math.log(goal_diff + 1, LOG_BASE)


In [11]:
df_teams = pdsql.sqldf("SELECT DISTINCT HomeTeam as TEAM FROM full_df")
df_teams["H_RATING"] = df_teams["A_RATING"] = 0.0

# set ratings as float
df_teams["H_RATING"] = df_teams["A_RATING"].astype(float)
df_stats = pd.DataFrame()
df_stats["RATING_DIFF"] = df_stats["RESULT"] = 0

for index, row in full_df.iterrows():
  ratings_diff= df_teams.loc[df_teams["TEAM"] == row["HomeTeam"], "H_RATING"].values[0] - df_teams.loc[df_teams["TEAM"] == row["AwayTeam"], "A_RATING"].values[0]
  result = 1 if row["FTHG"] > row["FTAG"] else 0 if row["FTHG"] == row["FTAG"] else -1

  df_stats.loc[index, "RATING_DIFF"] = ratings_diff
  df_stats.loc[index, "RESULT"] = result

  full_df.loc[index, "RATING_DIFF"] = ratings_diff
  full_df.loc[index, "H_RATING"] = df_teams.loc[df_teams["TEAM"] == row["HomeTeam"], "H_RATING"].values[0]
  full_df.loc[index, "A_RATING"] = df_teams.loc[df_teams["TEAM"] == row["AwayTeam"], "A_RATING"].values[0]
  
  new_elos: Tuple[int, int, int, int] = get_new_pi_rating(
    df_teams.loc[df_teams["TEAM"] == row["HomeTeam"], "H_RATING"].values[0],
    df_teams.loc[df_teams["TEAM"] == row["HomeTeam"], "A_RATING"].values[0],
    df_teams.loc[df_teams["TEAM"] == row["AwayTeam"], "H_RATING"].values[0],
    df_teams.loc[df_teams["TEAM"] == row["AwayTeam"], "A_RATING"].values[0],
    row["FTHG"],
    row["FTAG"]
    )
                                                 
  df_teams.loc[df_teams["TEAM"] == row["HomeTeam"], "H_RATING"] = new_elos[0]
  df_teams.loc[df_teams["TEAM"] == row["HomeTeam"], "A_RATING"] = new_elos[1]
  df_teams.loc[df_teams["TEAM"] == row["AwayTeam"], "H_RATING"] = new_elos[2]
  df_teams.loc[df_teams["TEAM"] == row["AwayTeam"], "A_RATING"] = new_elos[3]


df_teams.head(100)

,TEAM,H_RATING,A_RATING
0,Aston Villa,-0.301988,-0.502932
1,Everton,0.612622,0.112032
2,Fulham,-0.311326,-0.830051
3,Man City,1.628984,1.344440
4,Middlesbrough,-0.223930,-0.532981
5,Portsmouth,-0.269673,-0.582120
6,Sunderland,-0.266579,-0.545350
7,West Ham,0.359789,0.040228
8,Arsenal,0.924109,0.395097
9,Wigan,-0.186209,-0.329036


In [12]:
# Write full_df to csv
full_df.to_csv("Datasets/premier/full_df.csv", index=False)

In [13]:
BINS = 28
# Minimum and maximum rating diffs
min_rating_diff = df_stats["RATING_DIFF"].min()
max_rating_diff = df_stats["RATING_DIFF"].max()
print(df_stats["RATING_DIFF"].min())
print(df_stats["RATING_DIFF"].max())

# Create bins by their left edge
bins = np.linspace(min_rating_diff, max_rating_diff, BINS)
# take into account the left most and right most bins
bins[0] = bins[0] - 2
bins[-1] = bins[-1] + 2

# Merge bins so that each bin has at least 20 observations
while True:
  hist, bin_edges = np.histogram(df_stats["RATING_DIFF"], bins=bins)
  if (hist.min() > 20):
    break
  else:
    bins = np.delete(bins, np.argmin(hist) + 1)

# print bins and count ordered by bin left value
print("Bins:")
for i in range(len(bins) - 1):
  print(f"{bins[i]} - {bins[i + 1]}: {hist[i]}")


df_bins = pd.DataFrame()
df_bins["BIN_LEFT"] = bins[:-1]
df_bins["BIN_RIGHT"] = bins[1:]
df_bins["COUNT"] = hist

# Calculate empirical probability of home win, draw and away win for each bin'
df_bins["H_WINS"] = df_bins["DRAWS"] = df_bins["A_WINS"] = 0.0

for index, row in df_bins.iterrows():
  df_bin = df_stats.loc[(df_stats["RATING_DIFF"] >= row["BIN_LEFT"]) & (df_stats["RATING_DIFF"] < row["BIN_RIGHT"])]
  df_bins.loc[index, "H_WINS"] = df_bin.loc[df_bin["RESULT"] == 1].shape[0] / df_bin.shape[0]
  df_bins.loc[index, "DRAWS"] = df_bin.loc[df_bin["RESULT"] == 0].shape[0] / df_bin.shape[0]
  df_bins.loc[index, "A_WINS"] = df_bin.loc[df_bin["RESULT"] == -1].shape[0] / df_bin.shape[0]

df_bins.head(100)



-1.954136180568791
2.383428583087704
Bins:
-3.954136180568791 - -1.3115339933604213: 45
-1.3115339933604213 - -1.1508834465583289: 59
-1.1508834465583289 - -0.9902328997562365: 133
-0.9902328997562365 - -0.8295823529541442: 185
-0.8295823529541442 - -0.6689318061520517: 226
-0.6689318061520517 - -0.5082812593499593: 236
-0.5082812593499593 - -0.3476307125478668: 304
-0.3476307125478668 - -0.18698016574577458: 344
-0.18698016574577458 - -0.02632961894368213: 477
-0.02632961894368213 - 0.1343209278584101: 632
0.1343209278584101 - 0.29497147466050255: 567
0.29497147466050255 - 0.455622021462595: 512
0.455622021462595 - 0.6162725682646875: 399
0.6162725682646875 - 0.7769231150667799: 313
0.7769231150667799 - 0.9375736618688724: 285
0.9375736618688724 - 1.0982242086709648: 254
1.0982242086709648 - 1.2588747554730573: 213
1.2588747554730573 - 1.4195253022751493: 197
1.4195253022751493 - 1.5801758490772417: 142
1.5801758490772417 - 1.7408263958793342: 112
1.7408263958793342 - 1.90147694268142

,BIN_LEFT,BIN_RIGHT,COUNT,H_WINS,DRAWS,A_WINS
0,-3.954136,-1.311534,45,0.111111,0.133333,0.755556
1,-1.311534,-1.150883,59,0.118644,0.152542,0.728814
2,-1.150883,-0.990233,133,0.165414,0.172932,0.661654
3,-0.990233,-0.829582,185,0.167568,0.194595,0.637838
4,-0.829582,-0.668932,226,0.221239,0.247788,0.530973
5,-0.668932,-0.508281,236,0.262712,0.241525,0.495763
6,-0.508281,-0.347631,304,0.282895,0.259868,0.457237
7,-0.347631,-0.186980,344,0.343023,0.255814,0.401163
8,-0.186980,-0.026330,477,0.373166,0.295597,0.331237
9,-0.026330,0.134321,632,0.422468,0.292722,0.284810


In [14]:
# write probabilities to csv
df_bins.to_csv("Datasets/pi_rating_probabilities.csv", index=False)